In [2]:
#! python3
from bs4 import BeautifulSoup
import requests
import re
import time
import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

#登陆豆瓣
login_url ='https://accounts.douban.com/login'
driver = webdriver.Chrome()
driver.get(login_url)
account = driver.find_element_by_id('email')
account.clear()#清楚框框里的字，下同
account.send_keys('yuchijing@126.com')#真实账号被隐藏，请换成自己实际的账号
password =driver.find_element_by_id('password')
password.clear()
password.send_keys('gqns7788,.')#真实密码被隐藏，请换成自己实际的密码


In [7]:
captcha = driver.find_element_by_id('captcha_image')
if captcha:#如果抓到验证码了，恭喜你，需要打码
    captcha_field = driver.find_element_by_id('captcha_field')
    captcha_field.clear()
    captcha_field.send_keys(input('captcha is:'))#验证码手动挡
    account.send_keys(Keys.RETURN)
else:#如果没有就直接回车吧
    account.send_keys(Keys.RETURN)

df = pd.read_csv('user_list.csv')
def get_info(soup):
    a = str(soup)
    pattern = re.compile('共同的喜好\((\d+)',re.S)
    same_num = (re.findall(pattern,a))
    if len(same_num)>0:
        same_like = int(same_num[0])
    else:
        same_like = 0

    b = soup.find_all("h2")
    text = ""
    for i in b:
        if "电影" in str(i):
            text = str(i)
    pattern2 = re.compile('target.+>(\d+\S+)部看过',re.S)
    movie_num = re.findall(pattern2,text)

    if len(movie_num)>0:
        num_movies = int(movie_num[0])
    else:
        num_movies = 0

    if same_like > 0 and num_movies > 0 :
        return same_like,num_movies,same_like/num_movies
    else:
        return 0,0,0

df_user = pd.DataFrame(columns=['url', 'samelike', 'watched', 'rate_self'])

count = 0
for user in df["User"]:
    count = count + 1
    print(user)
    print("现在爬到第" + str(count) + "个用户")
    try:
        driver.get(user)
        html=driver.page_source
        soup=BeautifulSoup(html,'lxml')
        same_like,watched,rate_self = get_info(soup)

        df_user.loc[-1] = [user,same_like,watched,rate_self]  # adding a row
        df_user.index = df_user.index + 1  # shifting index
        df_user = df_user.sort_index()  # sorting by index
        time.sleep(5)
        df_user.to_csv('user_rate2.csv', encoding='utf-8')
    except:
        print("出问题啦！")
        break

captcha is:future
https://www.douban.com/people/richer725
现在爬到第1个用户
https://www.douban.com/people/49277454
现在爬到第2个用户
https://www.douban.com/people/150148834
现在爬到第3个用户
https://www.douban.com/people/161272245
现在爬到第4个用户
出问题啦！
